In [1]:
from meteostream import GefsClient
import xarray as xr

In [2]:
client = GefsClient(grib_dir="/home/tylerradebaugh/data/grib")

In [3]:
client.check_download = True
client.process_gefs()

Processing /home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f000.grib2...
Target idx: 5, Name:swh


ValueError: dimension 'valid_time' already exists as a scalar variable

In [12]:
ds = client._merge_data()
ds 

<xarray.Dataset> Size: 33GB
Dimensions:    (valid_time: 567, latitude: 721, longitude: 1440)
Coordinates:
    latitude   (latitude, longitude) float64 8MB 90.0 90.0 90.0 ... -90.0 -90.0
    longitude  (latitude, longitude) float64 8MB 0.0 0.25 0.5 ... 359.5 359.8
    ref_time   datetime64[ns] 8B 2025-02-04T18:00:00
Dimensions without coordinates: valid_time
Data variables:
    sigCww4    (valid_time, latitude, longitude) float64 5GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    sigCww5.5  (valid_time, latitude, longitude) float64 5GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    sigCww7    (valid_time, latitude, longitude) float64 5GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    sigCww9    (valid_time, latitude, longitude) float64 5GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wsBF10     (valid_time, latitude, longitude) float64 5GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wsBF8      (valid_time, latitude, longitude) float64 5GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    wsBF9      (valid_time, latitude, longitude) float64 5GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

In [3]:
import os

df = client.gefs_metadata()
# Filter the init index for the var_name
filtered_df = df.loc[client.idx_list]
sel_columns = filtered_df[['var_name', 'grb_shortName']]

# Convert to a dictionary with the idx_ID as keys and (var_name, grb_shortName) as values
var_dict = sel_columns.to_dict('index')
var_dict

{5: {'var_name': 'sigCww4', 'grb_shortName': 'swh'},
 6: {'var_name': 'sigCww5.5', 'grb_shortName': 'swh'},
 7: {'var_name': 'sigCww7', 'grb_shortName': 'swh'},
 8: {'var_name': 'sigCww9', 'grb_shortName': 'swh'},
 38: {'var_name': 'wsBF8', 'grb_shortName': 'ws'},
 39: {'var_name': 'wsBF9', 'grb_shortName': 'ws'},
 40: {'var_name': 'wsBF10', 'grb_shortName': 'ws'}}

In [5]:
datasets = []

# Loop through all variables
for idx, values in var_dict.items():
    var_name, grb_shortName = values['var_name'], values['grb_shortName']

    # Get a list of all GRIB2 files for the current variable
    grib_files = sorted(os.path.join(client.grib_dir, f) for f in os.listdir(client.grib_dir) if f.startswith(f"GEFS_{var_name}"))

    # Open all GRIB2 files for this variable and create a dataset
    with xr.open_mfdataset(
        grib_files,
        combine='nested',
        concat_dim='valid_time',
        engine='cfgrib',
        backend_kwargs={"indexpath": None}
    ) as ds:
        # Rename the variable for xarray to merge datasets effectively
        ds = ds.rename({grb_shortName: var_name})

        # Drop unnecessary variables and rename dimensions
        ds = ds.drop_vars('step', errors='ignore').rename({'time': 'ref_time'})
        print(f"Swapping vars: {grb_shortName} with {var_name}")

        # Append the dataset to the list
        datasets.append(ds)

# Concatenate all datasets along the `valid_time` dimension
final_ds = xr.concat(datasets, dim='valid_time')

# Print the final combined dataset
print(final_ds)



: 

In [6]:
concat_ds = xr.concat(datasets, dim='valid_time')
concat_ds

<xarray.Dataset> Size: 16GB
Dimensions:     (valid_time: 567, latitude: 721, longitude: 1440)
Coordinates:
    ref_time    (valid_time) datetime64[ns] 5kB 2025-02-04T12:00:00 ... 2025-...
    surface     float64 8B 0.0
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.3 359.5 359.8
  * valid_time  (valid_time) datetime64[ns] 5kB 2025-02-04T12:00:00 ... 2025-...
Data variables:
    sigCww4     (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    sigCww5.5   (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    sigCww7     (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    sigCww9     (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    wsBF8       (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    wsBF9       (valid_time, latitude, longitude) float32 2GB nan nan ... nan
    wsBF10      (valid_time, latitude, longitude) float32 2GB nan nan ... nan
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2025-02-04T21:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
file = "/home/tylerradebaugh/data/grib/gefs.wave.t18z.prob.global.0p25.f012.grib2"

In [8]:
import numpy as np
import pygrib

output = "testfile.nc"
var_name = "variable_name"  # Replace with your actual variable name


with pygrib.open(file) as grbs:
    grb = grbs.message(5)
    data = grb.values
    lats, lons = grb.latlons()
    valid_time = grb.validDate   # Valid time as datetime object
    ref_time = grb.analDate      # Reference (analysis) time as datetime object

    data = np.expand_dims(data, axis=0)

    print(data.shape)

    # # Create a DataArray with time as an additional dimension
    data_array = xr.DataArray(
        data,  
        coords={
            "latitude": (["latitude", "longitude"], lats),
            "longitude": (["latitude", "longitude"], lons),
            "valid_time": valid_time,
            "ref_time": ref_time
        },
        dims=["valid_time", "latitude", "longitude"],
        name = var_name,
        attrs={
            
            "Description": df.loc[5]['parameter']
            }
    )

    print(data_array)


(1, 721, 1440)
<xarray.DataArray 'variable_name' (valid_time: 1, latitude: 721, longitude: 1440)> Size: 8MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    latitude    (latitude, longitude) float64 8MB 90.0 90.0 90.0 ... -90.0 -90.0
    longitude   (latitude, longitude) float64 8MB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time  datetime64[ns] 8B 2025-02-05T06:00:00
    ref_time    datetime64[ns] 8B 2025-02-04T18:00:00
Attributes:
    Description:  Sig. ht. of combined wind waves and swell


In [13]:
import xarray as xr

with xr.open_dataset("/home/tylerradebaugh/data/grib/GEFS_wsBF10_gefs_idx40_f042.nc") as ds:
    print(ds)

<xarray.Dataset> Size: 25MB
Dimensions:    (latitude: 721, longitude: 1440, valid_time: 1)
Coordinates:
    latitude   (latitude, longitude) float64 8MB ...
    longitude  (latitude, longitude) float64 8MB ...
    ref_time   datetime64[ns] 8B ...
Dimensions without coordinates: valid_time
Data variables:
    wsBF10     (valid_time, latitude, longitude) float64 8MB ...


In [73]:
ds = xr.open_dataset("testfile.nc")
ds 

<xarray.Dataset> Size: 4MB
Dimensions:        (latitude: 721, longitude: 1440, valid_time: 1, ref_time: 1)
Coordinates:
  * latitude       (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float32 6kB 0.0 0.25 0.5 ... 359.3 359.5 359.8
  * valid_time     (valid_time) datetime64[ns] 8B 2025-02-05
  * ref_time       (ref_time) datetime64[ns] 8B 2025-02-04T12:00:00
Data variables:
    variable_name  (valid_time, latitude, longitude) float32 4MB ...

In [74]:
ds.close()

In [7]:
df = client.metadata
df 

,grb_shortName,parameter,unit,threshold,var_name
idx_ID,,,,,
1,swh,Sig. ht. of combined wind waves and swell,meters,Prob. > 0.6,sigCww0.6
2,swh,Sig. ht. of combined wind waves and swell,meters,Prob. > 1.0,sigCww1
3,swh,Sig. ht. of combined wind waves and swell,meters,Prob. > 2.0,sigCww2
4,swh,Sig. ht. of combined wind waves and swell,meters,Prob. > 3.0,sigCww3
5,swh,Sig. ht. of combined wind waves and swell,meters,Prob. > 4.0,sigCww4
...,...,...,...,...,...
67,mpts,Mean period of total swell,seconds,Prob. > 9.0,avgswper9
68,mpts,Mean period of total swell,seconds,Prob. > 11.0,avgswper11
69,mpts,Mean period of total swell,seconds,Prob. > 13.0,avgswper13


In [5]:
import dask
from dask.diagnostics import ProgressBar

def process_single_grib(input_filepath, output_dir, target_indices, var_dict, metadata):
    print(f'Processing {input_filepath}...')
    
    with pygrib.open(input_filepath) as grbs:
        for target_index in target_indices:
            # Extract the target parameter
            grb = grbs.message(target_index)
            print(f"Target idx: {target_index}, Name: {grb.shortName}")

            # Find matching variable
            if target_index in var_dict:
                values = var_dict[target_index]
                var_name = values['var_name']
                grb_name = values['grb_shortName']
                data = grb.values
                data_3d = np.expand_dims(data, axis=0)  # Add time dimension
                lats, lons = grb.latlons()
                valid_time = grb.validDate
                ref_time = grb.analDate
                df = metadata.loc[target_index]

                # Check if the extracted parameter matches the expected shortName
                if grb.shortName == grb_name:
                    output_filename = f"GEFS_{var_name}_{os.path.basename(input_filepath).split('.')[0]}_idx{target_index}.nc"
                    output_filepath = os.path.join(output_dir, output_filename)

                    # Create a DataArray and save it as NetCDF
                    data_array = xr.DataArray(
                        data_3d,
                        coords={
                            "latitude": (["latitude"], lats[:, 0]),
                            "longitude": (["longitude"], lons[0, :]),
                            "valid_time": (["valid_time"], [np.datetime64(valid_time)]),
                        },
                        dims=["valid_time", "latitude", "longitude"],
                        name=var_name,
                        attrs={
                            "grb_shortName": grb_name,
                            "grb_Param_Index": target_index,
                            "Description": df['parameter'],
                            "units": df['unit'],
                            "Limit": df['threshold'],
                            "ref_time": ref_time.isoformat(),
                        }
                    )

                    # Write to NetCDF using Dask
                    data_array.to_netcdf(output_filepath, compute=False)
                    print(f"Scheduled to save: {output_filepath}")

# Function to handle all GRIB files in parallel
def process_all_grib_files_parallel(grib_dir, output_dir, idx_list, var_dict, metadata):
    tasks = []

    # Loop through all GRIB files and create Dask tasks
    for filename in sorted(os.listdir(grib_dir)):
        if filename.startswith("gefs.wave"):
            input_filepath = os.path.join(grib_dir, filename)
            tasks.append(dask.delayed(process_single_grib)(input_filepath, output_dir, idx_list, var_dict, metadata))

    # Execute all tasks in parallel
    with ProgressBar():
        dask.compute(*tasks)


In [10]:
import os
import pygrib
import xarray as xr
import numpy as np

# Example usage
process_all_grib_files_parallel(
    grib_dir=client.grib_dir,
    output_dir=client.grib_dir,
    idx_list=client.idx_list,
    var_dict=client.var_dict,
    metadata=client.metadata
)

[                                        ] | 0% Completed | 233.64 us

[                                        ] | 0% Completed | 6.27 s ms


PermissionError: [Errno 13] Permission denied: '/home/tylerradebaugh/data/grib/GEFS_sigCww4_gefs_idx5.nc'